In [ ]:
import os
import sys 

if os.getcwd().endswith("notebooks"): os.chdir("..")

if "./src" not in sys.path: sys.path.append("./src")

In [ ]:
from os import getenv

from uuid import uuid4

from langchain_openai import ChatOpenAI
from langchain_core.prompts import PromptTemplate
from langchain_core.messages import HumanMessage, AIMessage, SystemMessage

from dotenv import load_dotenv

load_dotenv()

In [ ]:
from langgraph.checkpoint.memory import InMemorySaver

from langgraph.types import Command

from smart_investigator.foundation.agents.ticketing_agent_graph import create_ticketing_agent
from smart_investigator.foundation.agents.travel_agent_graph import create_travel_agent
from smart_investigator.foundation.agents.travel_agent_tool_agents import create_ticketing_agent_tool

In [ ]:
ticketing_llm = ChatOpenAI(
  api_key=getenv("OPENAI_API_KEY"),
  base_url="https://openrouter.ai/api/v1",
  model="gpt-4o-mini",
)

user_inputs = [
    "I want to go to Bali, Indonesia. 2 tickets.",
    "Sydney, Australia. Round-trip please. ",
    "yes. Proceed and invpoice me",
    "...",
    "yes, book the tickets for me.",
    "exit",
    "exit"
]
input_iter = iter(user_inputs)

greeting = "Hi! I'm your ticketing assistant. How can I help you today?\n"
print (greeting)
config={"configurable": {"thread_id": str(uuid4()) }}
agent_tkt = create_ticketing_agent(ticketing_llm, InMemorySaver())
while True:    
    content = next(input_iter) # input("Your response (or 'exit' to quit): ")
    if content.lower() in ['exit', 'quit']:
        print("Goodbye!")
        break
    human_message = HumanMessage(content=content)
    human_message.pretty_print()
    response = agent_tkt.invoke({"messages": [human_message], "loop_counter": 0}, config=config)
    while "__interrupt__" in response:
        response["messages"][-1].pretty_print()
        human_response = next(input_iter) # input (str(response["messages"][-1].content) + "\nYour response: ")
        response = agent_tkt.invoke(Command(resume=response), config=config)

    response["messages"][-1].pretty_print()

In [ ]:
user_inputs = [
    "I want to go to Bali. For 2 adults. Next week Wednesday.",
    "Departing from Sydney Australia, 30th November return. Nothing else. ",
    "yes, proceed with the booking",
    "ok,",
    "exit",
    "exit"
]
input_iter = iter(user_inputs)

def llm_factory():
    return ChatOpenAI(
      api_key=getenv("OPENAI_API_KEY"),
      base_url="https://openrouter.ai/api/v1",
      model="gpt-4o-mini",
    )

def checkpointer_factory():
    return InMemorySaver()

llm_travel = llm_factory()

def travel_agent_tool_factory(llm_factory, checkpointer_factory):
    agent_tkt = create_ticketing_agent(llm_factory(), checkpointer_factory())
    return [create_ticketing_agent_tool(agent_tkt=agent_tkt)]

agent_tools_travel = travel_agent_tool_factory(llm_factory, checkpointer_factory)

greeting = "Hi! I'm your travel assistant. How can I help you today?\n"
print (greeting)
config={"configurable": {"thread_id": str(uuid4()) }}
agent = create_travel_agent(llm_travel, InMemorySaver(), agent_tools=agent_tools_travel)
while True:    
    content = next(input_iter) # input("Your response (or 'exit' to quit):") # next(input_iter) # input("Your response (or 'exit' to quit): ")
    if content.lower() in ['exit', 'quit']:
        print("Goodbye!")
        break
    human_message = HumanMessage(content=content)
    human_message.pretty_print()
    response = agent.invoke({"messages": [human_message], "loop_counter": 0}, config=config)
    while "__interrupt__" in response:
        response["messages"][-1].pretty_print()
        print (f"__interrupt__ : {response['__interrupt__']}")
        human_response = next(input_iter) # input (str(response["messages"][-1].content) + "\nYour response: ")
        human_message = HumanMessage(content=human_response)
        human_message.pretty_print()
        response = agent.invoke(Command(resume=response), config=config)

    response["messages"][-1].pretty_print()

In [ ]:
import json
import uuid
from os import getenv

# MLflow imports required for the request/response objects
from mlflow.types.responses import ResponsesAgentRequest, ResponsesAgentResponse
from mlflow.pyfunc import PythonModelContext
from smart_investigator.foundation.agents.ticketing_agent import TicketingAgentWrapper

# --- Setup from original script ---
user_inputs = [
    "I want to go to Bali, Indonesia. 2 tickets.",
    "Sydney, Australia. Round-trip please. ",
    "yes. Proceed and invoice me",
    "yes, finalize the booking for me.", 
    "ok",
    "exit",
    "exit"
]
input_iter = iter(user_inputs)

# --- 1. Initialize the TicketingAgentWrapper ---
print("Initializing agent wrapper...")
agent_wrapper = TicketingAgentWrapper()

# Manually call load_context to initialize self.graph and self.llm
# Our simulated _load_llm_from_context will use env vars
# We pass `None` as the context, which our implementation handles.
agent_wrapper.load_context(context=PythonModelContext(artifacts={}, model_config={}))
print("Agent wrapper ready.")

# --- 2. Set up for ResponsesAgent interaction ---
# This ID will be used for all requests in this conversation
conversation_id = str(uuid.uuid4())

# This flag tracks if the agent is waiting for a resume input
is_waiting_for_resume = False

greeting = "Hi! I'm your ticketing assistant. How can I help you today?\n"
print(greeting)

# --- 3. Run the interaction loop ---
while True:
    try:
        # Get the next user input
        content = next(input_iter) 
        print(f"\nUser: {content}")
    except StopIteration:
        print("\n--- End of user inputs ---")
        break

    # Check for exit condition
    if content.lower() in ['exit', 'quit']:
        print("Goodbye!")
        break

    # --- 4. Build the ResponsesAgentRequest ---
    request_input = []
    request_custom_inputs = {}

    if is_waiting_for_resume:
        # If we were interrupted, this input is the "resume" signal
        # We serialize it to JSON as required by our design
        request_custom_inputs["__resume__"] = json.dumps(content)
        # request.input is empty, as the payload is in custom_inputs
    else:
        # This is a new, normal message
        request_input = [{"role": "user", "content": content}]

    # Create the request object
    request = ResponsesAgentRequest(
        input=request_input,
        context={"conversation_id": conversation_id},
        custom_inputs=request_custom_inputs
    )

    # --- 5. Call the wrapper's predict method ---
    # This single call replaces the .invoke() and the inner interrupt loop
    try:
        response: ResponsesAgentResponse = agent_wrapper.predict(request)
    except Exception as e:
        print(f"An error occurred during agent prediction: {e}")
        raise e

    # --- 6. Process the ResponsesAgentResponse ---
    
    # The agent's message (final or interrupt) is in the output
    if response.output:
        agent_message = response.output[0].content
        print(f"Agent: {agent_message}")
    else:
        print("Agent: [No response output]")

    # --- 7. Update state for the next loop ---
    if "__interrupt__" in response.custom_outputs:
        # The agent has paused. The next input will be a resume.
        print("[Agent is waiting for more information]")
        is_waiting_for_resume = True
    else:
        # The agent finished. The next input will be a new message.
        is_waiting_for_resume = False
        
        # If the graph finishes without an interrupt *or* a final message,
        # we can end the loop.
        if not response.output:
            print("[Agent finished processing]")
            break

In [ ]:
import json
import uuid
from os import getenv

# MLflow imports required for the request/response objects
from mlflow.types.responses import ResponsesAgentRequest, ResponsesAgentResponse
from mlflow.pyfunc import PythonModelContext
from smart_investigator.foundation.agents.travel_agent import TravelAgentWrapper


# --- 1. Initialize the TravelAgentWrapper ---
print("Initializing agent wrapper...")
agent_wrapper = TravelAgentWrapper()

# Manually call load_context to initialize self.graph and self.llm
# Our simulated _load_llm_from_context will use env vars
# We pass `None` as the context, which our implementation handles.
agent_wrapper.load_context(context=PythonModelContext(artifacts={}, model_config={}))
print("Agent wrapper ready.")

# --- 2. Set up for ResponsesAgent interaction ---
# This ID will be used for all requests in this conversation
conversation_id = str(uuid.uuid4())

# This flag tracks if the agent is waiting for a resume input
is_waiting_for_resume = False



In [ ]:

# --- Setup from original script ---
user_inputs = [
    "I want to go to Bali. For 2 adults. Next week Wednesday.",
    "Departing from Sydney Australia, 30th November return. Nothing else. ",
    "yes, proceed with the booking",
    "ok,",
    "exit",
    "exit"
]
input_iter = iter(user_inputs)

greeting = "Hi! I'm your travel agent. How can I help you today?\n"
print(greeting)

# --- 3. Run the interaction loop ---
while True:
    try:
        # Get the next user input
        content = next(input_iter) 
        print(f"\nUser: {content}")
    except StopIteration:
        print("\n--- End of user inputs ---")
        break

    # Check for exit condition
    if content.lower() in ['exit', 'quit']:
        print("Goodbye!")
        break

    # --- 4. Build the ResponsesAgentRequest ---
    request_input = []
    request_custom_inputs = {}

    if is_waiting_for_resume:
        # If we were interrupted, this input is the "resume" signal
        # We serialize it to JSON as required by our design
        request_custom_inputs["__resume__"] = json.dumps(content)
        # request.input is empty, as the payload is in custom_inputs
    else:
        # This is a new, normal message
        request_input = [{"role": "user", "content": content}]

    # Create the request object
    request = ResponsesAgentRequest(
        input=request_input,
        context={"conversation_id": conversation_id},
        custom_inputs=request_custom_inputs
    )

    # --- 5. Call the wrapper's predict method ---
    # This single call replaces the .invoke() and the inner interrupt loop
    try:
        response: ResponsesAgentResponse = agent_wrapper.predict(request)
    except Exception as e:
        print(f"An error occurred during agent prediction: {e}")
        raise e

    # --- 6. Process the ResponsesAgentResponse ---
    
    # The agent's message (final or interrupt) is in the output
    if response.output:
        agent_message = response.output[0].content
        print(f"Agent: {agent_message}")
    else:
        print("Agent: [No response output]")

    # --- 7. Update state for the next loop ---
    if "__interrupt__" in response.custom_outputs:
        # The agent has paused. The next input will be a resume.
        print("[Agent is waiting for more information]")
        is_waiting_for_resume = True
    else:
        # The agent finished. The next input will be a new message.
        is_waiting_for_resume = False
        
        # If the graph finishes without an interrupt *or* a final message,
        # we can end the loop.
        if not response.output:
            print("[Agent finished processing]")
            break

In [ ]:
from uuid import uuid4
from pprint import pprint
config={"configurable": {"thread_id": str(uuid4()) }}
for chunk in agent_wrapper.graph.stream({"messages": ["I want to go on a vacation. You should ask for help."], "loop_counter": -999}, config=config, stream_mode="values"):
    pprint(chunk)
request = ResponsesAgentRequest(
        input=[{"role": "user", "content": "I want to go on a vacation. You should ask for help."}],
        context={"conversation_id": str(uuid4())},
        custom_inputs={}
    )

for chunk in agent_wrapper.predict_stream(request):
    pprint(chunk)